# ∞-Sector Convergence vs Operator Strength (α-Comparison)

This notebook compares ∞-sector-style convergence for different values of the scaling parameter $\alpha$.

Given the pseudoinverse of the DLSFH Laplacian, $L^+$, we define
$$ K_{\alpha} = \alpha \frac{L^+}{\lVert L^+ \rVert_2}, \quad \alpha \in (0,1), $$
so that $\lVert K_{\alpha} \rVert_2 \leq \alpha$.

For each $\alpha$, we form partial sums
$$ S_N^{(\alpha)} = \sum_{n=0}^N K_{\alpha}^n, $$
use a fixed cutoff $N_{\max}$ as an approximation to the ∞-sector limit, and track
$$ \mathrm{err}_2^{(\alpha)}(N) = \lVert S_N^{(\alpha)} - S_{N_{\max}}^{(\alpha)} \rVert_2. $$

We then plot $\mathrm{err}_2^{(\alpha)}(N)$ on a common log-scale axis to visualize how the convergence
depends on $\alpha$ (i.e., on the effective spectral radius of $K_{\alpha}$).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load pseudoinverse
L_pinv_path = "../data/Delta20_pinv.npy"  # adjust if needed
L_pinv = np.load(L_pinv_path)
print("L^+ shape:", L_pinv.shape)

# Operator norm of L^+
op_norm = np.linalg.norm(L_pinv, ord=2)
print("Operator norm ||L^+||_2 =", op_norm)

# Alpha values to compare
alphas = [0.3, 0.5, 0.7, 0.9]
print("Comparing α values:", alphas)

N_max = 50
dim = L_pinv.shape[0]
I = np.eye(dim)


## Compute error curves for each α

For each $\alpha$:
- Construct $K_{\alpha}$
- Build $S_{N_{\max}}^{(\alpha)}$
- Compute $\mathrm{err}_2^{(\alpha)}(N)$ for $N = 0, \dots, N_{\max}$


In [ ]:
Ns = list(range(N_max + 1))
errors_by_alpha = {}

for alpha in alphas:
    print("\n[α =", alpha, "]")
    # build K_alpha
    K = alpha * (L_pinv / op_norm)
    print("Approximate ||K_α||_2 <=", alpha)

    # build S_Nmax^{(α)}
    S_Nmax = np.zeros_like(K)
    current = I.copy()
    for n in range(N_max + 1):
        S_Nmax += current
        current = current @ K

    # now generate partial sums and errors
    errors_2 = []
    S_N = np.zeros_like(K)
    current = I.copy()

    for N in Ns:
        if N == 0:
            S_N = current.copy()
        else:
            S_N += current
        diff = S_N - S_Nmax
        err2 = np.linalg.norm(diff, ord=2)
        errors_2.append(err2)
        current = current @ K

    errors_by_alpha[alpha] = np.array(errors_2)
    print("  initial error (N=0):", errors_2[0])
    print("  final error   (N=N_max):", errors_2[-1])


## Log-scale comparison plot

We now plot $\mathrm{err}_2^{(\alpha)}(N)$ for all $\alpha$ on the same set of axes.


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

for alpha in alphas:
    ax.plot(Ns, errors_by_alpha[alpha], marker="o", label=fr"α = {alpha}")

ax.set_yscale("log")
ax.set_xlabel("N (partial sum index)")
ax.set_ylabel(r"$||S_N^{(α)} - S_{N_{max}}^{(α)}||_2$")
ax.set_title(r"∞-sector convergence vs $α$ (operator strength)")
ax.grid(True, which="both", ls=":", alpha=0.6)
ax.legend()

plt.tight_layout()
plt.show()

### Interpretation

- Smaller $α$ (smaller effective spectral radius) leads to faster decay of the error curve.
- As $α$ approaches 1, convergence becomes slower, reflecting the behavior expected from a geometric series.

In the context of VID, this toy model illustrates how the strength of the underlying ∞-sector operator controls
the rate at which the nonperturbative tower converges to a stable geometric object.
